In [1]:
import pandas as pd
import numpy as np
import operator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
import xgboost as xgb

#讀檔
f = open('training_label.txt', mode='r', encoding='utf-8')
f2 = open('testing_label.txt', mode='r', encoding='utf-8')

x = []
train = []
test = []

#train傳換為list(2維)，且只取前10000筆
for line in f:
     x.append(line.strip('\n'))
for i in range(10000):
    train.append(x[i].split(' +++$+++ '))

#test傳換為list(2維)    
for line in f2:
    if line == '\n':
        continue
    test.append(line.strip('\n').split('#####'))
    
#以下為list的降維：
#list轉str
temp1 = str(train)
#替換'[' ']'
temp1 = temp1.replace('[','')
temp1 = temp1.replace(']','')
#轉list
train = list(eval(temp1))

#test同上
temp2 = str(test)
temp2 = temp2.replace('[','')
temp2 = temp2.replace(']','')
test = list(eval(temp2))


In [2]:
#將train的label與data分成2 list
train_emotion = []
train_words = []
for i in range(20000):
    if (i%2 == 0):
        train_emotion.append(train[i])
    else:
        train_words.append(train[i])
#Dataframe
df_train = pd.DataFrame(list(zip(train_emotion, train_words)), columns =['emotion', 'words'])
df_train

,emotion,words
0,1,are wtf ... awww thanks !
1,1,leavingg to wait for kaysie to arrive myspacin...
2,0,i wish i could go and see duffy when she comes...
3,1,i know eep ! i can ' t wait for one more day ....
4,0,so scared and feeling sick . fuck ! hope someo...
...,...,...
9995,0,i know the feeling !! but don ' t be hun ! eve...
9996,1,i was just asked today if i was a conservative...
9997,0,cubs white soxs game rained out ... of all nig...
9998,1,props to your mom she did a great job !


In [3]:
#將test的label與data分成2 list
test_emotion = []
test_words = []
for i in range(180):
    if (i%2 == 0):
        test_emotion.append(test[i])
    else:
        test_words.append(test[i])
#Dataframe        
df_test = pd.DataFrame(list(zip(test_emotion, test_words)), columns =['emotion', 'words'])
df_test

,emotion,words
0,0,don ' t you hate it when you hang on to a seem...
1,0,ok so never went to the movies because friend ...
2,1,can ' t wait to see diversity ' s performance !
3,1,i love britney spears haha joey this is what u...
4,1,1 more day !
...,...,...
85,1,shower time !!! then more shopping for my new ...
86,1,awwwwww too cute ! makes me miss my westie bac...
87,0,thanks . lol . sadly i am 31 !! waaahhhhh
88,1,thanks ! i am listening to her on lala . com r...


In [14]:
#去除停用字，停用字庫網路搜尋而得
words = train_words+test_words
stopwords=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "ah", "20", "5977", "0347", "18", "any", "70"]
cv = CountVectorizer(stop_words=stopwords) #將文字中的詞語轉換為詞頻矩陣 
x_train = cv.fit_transform(words) #計算個詞語出現的次數
print(x_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [5]:
#將詞頻矩陣統計成TF-IDF值 
transformer = TfidfTransformer()  
x_tfidf = transformer.fit_transform(x_train)   
x_tfidf = x_tfidf.toarray()
x_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
#合併train與test的label成為一個list y
y = train_emotion + test_emotion

In [7]:
#用AdaBoost模型建模
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_tfidf, y)  

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [9]:
#印出Accuracy、Precision、Recall、F-1分數
pred = clf.predict(x_tfidf[10000:10090])#切出label的tfidf做prediction
print("AdaBoost模型-")
print(classification_report(df_test['emotion'], pred))

AdaBoost模型-
Accuracy =  0.7001982160555005
              precision    recall  f1-score   support

           0       0.76      0.59      0.67        37
           1       0.75      0.87      0.81        53

    accuracy                           0.76        90
   macro avg       0.76      0.73      0.74        90
weighted avg       0.76      0.76      0.75        90



In [12]:
#用XGboost模型建模並印出Accuracy、Precision、Recall、F-1分數
from xgboost import XGBClassifier
XGB = XGBClassifier()
XGB.fit(x_tfidf, y)
pred = XGB.predict(x_tfidf[10000:10090])#切出label的tfidf做prediction
print("XGBoost模型-")
print(classification_report(df_test['emotion'], pred))

XGBoost模型-
              precision    recall  f1-score   support

           0       0.73      0.51      0.60        37
           1       0.72      0.87      0.79        53

    accuracy                           0.72        90
   macro avg       0.72      0.69      0.69        90
weighted avg       0.72      0.72      0.71        90

